In [7]:
# run this to shorten the data import from the files
path_data = '/home/nero/Documents/Estudos/DataCamp/Python/Image_Processing_with_Keras_in_Python/datasets/'
import pandas as pd
import numpy as np


In [8]:
img_rows, img_cols = 28,28

In [9]:
# exercise 01

"""
Creating a deep learning network

A deep convolutional neural network is a network that has more than one layer. Each layer in a deep network receives its input from the preceding layer, with the very first layer receiving its input from the images used as training or test data.

Here, you will create a network that has two convolutional layers.
"""

# Instructions

"""

    The first convolutional layer is the input layer of the network. This should have 15 units with kernels of 2 by 2 pixels. It should have a 'relu' activation function. It can use the variables img_rows and img_cols to define its input_shape.
    The second convolutional layer receives its inputs from the first layer. It should have 5 units with kernels of 2 by 2 pixels. It should also have a 'relu' activation function.

"""

# solution

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

model = Sequential()

# Add a convolutional layer (15 units)
model.add(Conv2D(15, input_shape = (img_rows, img_cols, 1), kernel_size =2, activation = 'relu'))


# Add another convolutional layer (5 units)
model.add(Conv2D(5, kernel_size = 2, activation = 'relu'))

# Flatten and feed to output layer
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

#----------------------------------#

# Conclusion

"""
Congratulations!! You built a network with multiple convolution layers.
"""

'\nCongratulations!! You built a network with multiple convolution layers.\n'

In [10]:
data = pd.read_csv('/home/nero/Documents/Estudos/DataCamp/Python/courses/Image_Processing_with_Keras_in_Python/datasets/clothing_mnist/fashion-mnist_train.csv')

In [11]:
train_data = data.drop(columns='label').iloc[:50, :].values.reshape(50,28,28,1)

train_labels = data['label'][:50]


train_data.shape, train_labels.shape

((50, 28, 28, 1), (50,))

In [12]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels)

train_labels.shape

(50, 10)

In [13]:
test_data = data.drop(columns=['label']).iloc[50:60, :].values.reshape(10,28,28,1)
test_labels = to_categorical(data['label'][50:60])

test_data.shape, test_labels.shape

((10, 28, 28, 1), (10, 10))

In [14]:
# exercise 02

"""
Train a deep CNN to classify clothing images

Training a deep learning model is very similar to training a single layer network. Once the model is constructed (as you have done in the previous exercise), the model needs to be compiled with the right set of parameters. Then, the model is fit by providing it with training data, as well as training labels. After training is done, the model can be evaluated on test data.

The model you built in the previous exercise is available in your workspace.
"""

# Instructions

"""

    Compile the model to use the categorical cross-entropy loss function and the Adam optimizer.
    Train the network with train_data for 3 epochs with batches of 10 images each.
    Use randomly selected 20% of the training data as validation data during training.
    Evaluate the model with test_data, use a batch size of 10.

"""

# solution

# Compile model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Fit the model to training data 
model.fit(train_data, train_labels, 
          validation_split=0.2, 
          epochs=3, batch_size=10)

# Evaluate the model on test data
model.evaluate(test_data, test_labels, batch_size=10)

#----------------------------------#

# Conclusion

"""
Accuracy calculated on the test data is not subject to overfitting.
"""

Epoch 1/3
4/4 [==============================] - 3s 358ms/step - loss: 64.2108 - accuracy: 0.1000 - val_loss: 35.5033 - val_accuracy: 0.0000e+00
Epoch 2/3
4/4 [==============================] - 0s 66ms/step - loss: 21.9195 - accuracy: 0.3000 - val_loss: 26.3906 - val_accuracy: 0.2000
Epoch 3/3
1/1 [==============================] - 0s 68ms/step - loss: 6.5399 - accuracy: 0.6000


'\nAccuracy calculated on the test data is not subject to overfitting.\n'

In [15]:
# exercise 03

"""
How many parameters in a deep CNN?

In this exercise, you will use Keras to calculate the total number of parameters along with the number of parameters in each layer of the network.

We have already provided code that builds a deep CNN for you.
"""

# Instructions

"""
Summarize the network, providing a count of the number of parameters.
"""

# solution

# CNN model
model = Sequential()
model.add(Conv2D(10, kernel_size=2, activation='relu', 
                 input_shape=(28, 28, 1)))
model.add(Conv2D(10, kernel_size=2, activation='relu'))
model.add(Flatten())
model.add(Dense(3, activation='softmax'))

# Summarize the model 
model.summary()

#----------------------------------#

# Conclusion

"""
This model has 20,743 parameters!
"""

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 27, 27, 10)        50        
                                                                 
 conv2d_5 (Conv2D)           (None, 26, 26, 10)        410       
                                                                 
 flatten_2 (Flatten)         (None, 6760)              0         
                                                                 
 dense_2 (Dense)             (None, 3)                 20283     
                                                                 
Total params: 20,743
Trainable params: 20,743
Non-trainable params: 0
_________________________________________________________________


'\nThis model has 20,743 parameters!\n'

In [17]:
im = pd.read_csv('/home/nero/Documents/Estudos/DataCamp/Python/courses/Image_Processing_with_Keras_in_Python/datasets/im_ch03.csv').drop(columns = ['Unnamed: 0']).values
im.shape

(128, 128)

In [18]:
# exercise 04

"""
Write your own pooling operation

As we have seen before, CNNs can have a lot of parameters. Pooling layers are often added between the convolutional layers of a neural network to summarize their outputs in a condensed manner, and reduce the number of parameters in the next layer in the network. This can help us if we want to train the network more rapidly, or if we don't have enough data to learn a very large number of parameters.

A pooling layer can be described as a particular kind of convolution. For every window in the input it finds the maximal pixel value and passes only this pixel through. In this exercise, you will write your own max pooling operation, based on the code that you previously used to write a two-dimensional convolution operation.
"""

# Instructions

"""

    Index into the input array (im) and select the right window.
    Find the maximum in this window.
    Allocate this into the right entry in the output array (result).

"""

# solution

# Result placeholder
result = np.zeros((im.shape[0]//2, im.shape[1]//2))

# Pooling operation
for ii in range(result.shape[0]):
    for jj in range(result.shape[1]):
        result[ii, jj] = np.max(im[ii*2:ii*2+2, jj*2:jj*2+2])

#----------------------------------#

# Conclusion

"""
The resulting image is smaller, but retains the salient features in every location
"""

'\nThe resulting image is smaller, but retains the salient features in every location\n'

In [19]:
# exercise 05

"""
Keras pooling layers

Keras implements a pooling operation as a layer that can be added to CNNs between other layers. In this exercise, you will construct a convolutional neural network similar to the one you have constructed before:

Convolution => Convolution => Flatten => Dense

However, you will also add a pooling layer. The architecture will add a single max-pooling layer between the convolutional layer and the dense layer with a pooling of 2x2:

Convolution => Max pooling => Convolution => Flatten => Dense

A Sequential model along with Dense, Conv2D, Flatten, and MaxPool2D objects are available in your workspace.
"""

# Instructions

"""

    Add an input convolutional layer (15 units, kernel size of 2, relu activation).
    Add a maximum pooling operation (pooling over windows of size 2x2).
    Add another convolution layer (5 units, kernel size of 2, relu activation).
    Flatten the output of the second convolution and add a Dense layer for output (3 categories, softmax activation).

"""

# solution

model = Sequential()

from tensorflow.keras.layers import MaxPool2D

# Add a convolutional layer
model.add(Conv2D(15, kernel_size=2, activation='relu', 
                 input_shape=(img_rows, img_cols, 1)))

# Add a pooling operation
model.add(MaxPool2D(2))

# Add another convolutional layer
model.add(Conv2D(5, kernel_size= 2, activation = 'relu'))

# Flatten and feed to output layer
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
model.summary()

#----------------------------------#

# Conclusion

"""
This model is even deeper, but has fewer parameters.
"""

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 27, 27, 15)        75        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 15)       0         
 )                                                               
                                                                 
 conv2d_7 (Conv2D)           (None, 12, 12, 5)         305       
                                                                 
 flatten_3 (Flatten)         (None, 720)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                7210      
                                                                 
Total params: 7,590
Trainable params: 7,590
Non-trainable params: 0
____________________________________________________

'\nThis model is even deeper, but has fewer parameters.\n'

In [20]:
# exercise 06

"""
Train a deep CNN with pooling to classify images

Training a CNN with pooling layers is very similar to training of the deep networks that y have seen before. Once the network is constructed (as you did in the previous exercise), the model needs to be appropriately compiled, and then training data needs to be provided, together with the other arguments that control the fitting procedure.

The following model from the previous exercise is available in your workspace:

Convolution => Max pooling => Convolution => Flatten => Dense
"""

# Instructions

"""

    Compile this model to use the categorical cross-entropy loss function and the Adam optimizer.
    Train the model for 3 epochs with batches of size 10.
    Use 20% of the data as validation data.
    Evaluate the model on test_data with test_labels (also batches of size 10).

"""

# solution

# Compile the model
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Fit to training data
model.fit(train_data, train_labels, epochs = 3, batch_size = 10, validation_split = 0.20)

# Evaluate on test data 
model.evaluate(test_data, test_labels, batch_size = 10)

#----------------------------------#

# Conclusion

"""
This model is also very accurate!
"""

Epoch 1/3
4/4 [==============================] - 3s 181ms/step - loss: 14.4677 - accuracy: 0.1250 - val_loss: 8.3953 - val_accuracy: 0.2000
Epoch 2/3
4/4 [==============================] - 0s 37ms/step - loss: 9.2980 - accuracy: 0.2250 - val_loss: 7.2033 - val_accuracy: 0.2000
Epoch 3/3
1/1 [==============================] - 0s 64ms/step - loss: 3.9203 - accuracy: 0.3000


'\nThis model is also very accurate!\n'

In [21]:
model.save('/home/nero/Documents/Estudos/DataCamp/Python/courses/Image_Processing_with_Keras_in_Python/datasets/full_model.hdf5')